# Calculate CO$_2$ emissions related to European electricity generation - Part 1 - Calculation

Calculate emission values related to public electricity generation for the EU27 countries plus Great Britain and Norway for the period from 1990 to 2018.

In [1]:
## Using Python 3.7
import pandas as pd    # using version '1.1.1' including the pyxlsb package version '1.0.8'

Required data files:

- data/eea/UNFCCC_v23.csv
    - Source: https://www.eea.europa.eu/data-and-maps/data/national-emissions-reported-to-the-unfccc-and-to-the-eu-greenhouse-gas-monitoring-mechanism-16
    - Direct link to source file: https://www.eea.europa.eu/data-and-maps/data/national-emissions-reported-to-the-unfccc-and-to-the-eu-greenhouse-gas-monitoring-mechanism-16/national-greenhouse-gas-inventories-ipcc-common-reporting-format-sector-classification/ascii-delimited-zip-2/at_download/file
    - Downloaded 2020-12-30 (modified by EEA last on 2020-12-21)
- data/eurostat-energy_balances-june_2020_edition/
    - Extracted zip file in this folder
    - Source: https://ec.europa.eu/eurostat/web/energy/data/energy-balances
    - Direct link to source file: https://ec.europa.eu/eurostat/documents/38154/4956218/Energy-Balances-April-2020-edition.zip/69da6e9f-bf8f-cd8e-f4ad-50b52f8ce616
    - Downloaded: 2021-02-01 (modified by Eurostat last in 2020-06-05)

In [2]:
## Settings
## Input data files
path_eea = 'data/eea/UNFCCC_v23.csv'
path_eurostat = 'data/eurostat-energy_balances-june_2020_edition'  # path to folder
## Output data files
path_output_elec_emis = 'data/results/elec_emissions.csv'
path_output_elec_emis_autoprod = 'data/results/elec_emission_incl_autoprod.csv'

In [3]:
## Read UNFCCC emission statistic provided by the EEA
countries_eea = ['FR', 'DE', 'UK', 'IT', 'ES', 'PL', 'SE', 'NL', 'BE', 'FI', 'CZ', 'DK', 'PT', 'RO', 'AT', 
                 'BG', 'EE', 'GR', 'LV', 'HU', 'IE', 'SK', 'LT', 'HR', 'LU', 'SI', 'CY', 'MT', 'NO'] # + ['IS', 'TR']
sector = '1.A.1.a - Public Electricity and Heat Production'
pol = 'CO2'  # 'All greenhouse gases - (CO2 equivalent)' or 'CO2'

df_eea = pd.read_csv(path_eea, encoding='latin-1', low_memory=False)
df_eea.loc[df_eea['Year'] == '1985-1987', 'Year'] = 1986
df_eea['Year'] = df_eea['Year'].astype(int)
df_eea = df_eea.set_index(['Country_code', 'Pollutant_name', 'Year', 'Sector_name']).sort_index()

eea_emissions = df_eea.loc[pd.IndexSlice[countries_eea,pol,:,sector]].reset_index().rename(columns={'Country_code':'country', 'Year':'year'}).set_index(['country','year']).emissions
eea_emissions.rename(index={'UK':'GB'}, inplace=True)
eea_emissions = eea_emissions[eea_emissions.index.get_level_values('year')>=1990]  # Not interested in years before 1990
eea_emissions = eea_emissions.sort_index()/1e3  # Convert Gg to Tg CO2 (teragrams)

In [4]:
eea_emissions.head()

country  year
AT       1990    11.079379
         1991    11.792010
         1992     8.601634
         1993     8.436119
         1994     8.712956
Name: emissions, dtype: float64

In [5]:
## Read Eurostat energy balances (takes a few minutes to complete)
countries = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 
             'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']

index = pd.MultiIndex.from_product([countries, range(1990,2019)], names=('country', 'year'))
columns = ['TI_EHG_MAPE_E', 'TI_EHG_MAPCHP_E', 'TI_EHG_MAPH_E', 
           'TI_EHG_APE_E', 'TI_EHG_APCHP_E', 'TI_EHG_APH_E', 
           'GHP_MAPCHP', 'GHP_MAPH', 'GHP_APCHP', 'GHP_APH']
eurostat_nrg_bal = pd.DataFrame(index=index, columns=columns)

for i, country in enumerate(countries):
    print('Started reading country file of ' + country + ', file '+str(i+1)+' of '+str(len(countries))+'.')
    for year in range(1990,2019):
        filename = '{}/{}-Energy-balance-sheets-June-2020-edition.xlsb'.format(path_eurostat, country)
        df_tmp = pd.read_excel(filename, str(year), engine='pyxlsb', header=4)
        df_tmp = df_tmp.rename(columns={'Unnamed: 7':'category'})[['category','Total']].set_index('category')
        eurostat_nrg_bal.loc[country,year] = df_tmp.loc[columns].squeeze()

eurostat_nrg_bal.rename(index={'UK':'GB', 'EL':'GR'}, inplace=True)  # Rename EL -> GR, UK -> GB.

Started reading country file of AT, file 1 of 29.
Started reading country file of BE, file 2 of 29.
Started reading country file of BG, file 3 of 29.
Started reading country file of CY, file 4 of 29.
Started reading country file of CZ, file 5 of 29.
Started reading country file of DE, file 6 of 29.
Started reading country file of DK, file 7 of 29.
Started reading country file of EE, file 8 of 29.
Started reading country file of EL, file 9 of 29.
Started reading country file of ES, file 10 of 29.
Started reading country file of FI, file 11 of 29.
Started reading country file of FR, file 12 of 29.
Started reading country file of HR, file 13 of 29.
Started reading country file of HU, file 14 of 29.
Started reading country file of IE, file 15 of 29.
Started reading country file of IT, file 16 of 29.
Started reading country file of LT, file 17 of 29.
Started reading country file of LU, file 18 of 29.
Started reading country file of LV, file 19 of 29.
Started reading country file of MT, file

In [6]:
eurostat_nrg_bal.head()

TI_EHG_MAPE_E TI_EHG_MAPCHP_E TI_EHG_MAPH_E TI_EHG_APE_E  \
country year                                                            
AT      1990       4157.48         1360.54       340.262      497.741   
        1991       4160.68         1508.66       387.578      572.263   
        1992       3798.16         1334.34       382.906      691.296   
        1993       3854.84         1454.72       425.626      722.143   
        1994       3932.05         1452.46       386.242      885.065   

             TI_EHG_APCHP_E TI_EHG_APH_E GHP_MAPCHP GHP_MAPH GHP_APCHP GHP_APH  
country year                                                                    
AT      1990        839.978        0.979    316.423  267.006    95.085   0.931  
        1991         933.56        0.982     391.97  301.591   112.473   1.027  
        1992        890.131        1.963    357.648  301.591   117.106    1.17  
        1993        935.681        0.986    371.167  355.594   140.179   1.003  
        1994        1082.65        0.986    421.324  312.625    105.45    0.86

In [7]:
## Create convenient products
eurostat_products = pd.DataFrame(index=eurostat_nrg_bal.index, columns=['MAP_EI', 'AP_EI', 'MAP_DHO', 'AP_DHO'])
eurostat_products['MAP_EI'] = eurostat_nrg_bal['TI_EHG_MAPE_E'] + eurostat_nrg_bal['TI_EHG_MAPCHP_E'] + eurostat_nrg_bal['TI_EHG_MAPH_E']
eurostat_products['AP_EI'] = eurostat_nrg_bal['TI_EHG_APE_E'] + eurostat_nrg_bal['TI_EHG_APCHP_E'] + eurostat_nrg_bal['TI_EHG_APH_E']
eurostat_products['MAP_DHO'] = eurostat_nrg_bal['GHP_MAPCHP'] + eurostat_nrg_bal['GHP_MAPH']
eurostat_products['AP_DHO'] = eurostat_nrg_bal['GHP_APCHP'] + eurostat_nrg_bal['GHP_APH']

In [8]:
eurostat_products.head()

MAP_EI    AP_EI  MAP_DHO   AP_DHO
country year                                    
AT      1990  5858.29   1338.7  583.429   96.016
        1991  6056.92   1506.8  693.561    113.5
        1992  5515.41  1583.39  659.239  118.276
        1993  5735.18  1658.81  726.761  141.182
        1994  5770.75   1968.7  733.949   106.31

In [9]:
## Compute emission values of electricity generation
elec_emission = eea_emissions * ((eurostat_products['MAP_EI'] - (eurostat_nrg_bal['GHP_MAPCHP']/0.9)) / (eurostat_products['MAP_EI'] + eurostat_products['AP_EI']))  # The final units are Tg CO2.
elec_emission[elec_emission<0] = 0
elec_emission.name = 'electricity-only emissions'

In [10]:
elec_emission.head()

country  year
AT       1990    8.47728
         1991    8.76388
         1992    6.20152
         1993    6.07298
         1994     5.9696
Name: electricity-only emissions, dtype: object

In [11]:
## Compute emission values of electricity generation including estimated contribution from autoproducers
elec_emission_incl_autoprod = eea_emissions * ((eurostat_products['MAP_EI'] - (eurostat_nrg_bal['GHP_MAPCHP']/0.9)) / (eurostat_products['MAP_EI'] + eurostat_products['AP_EI'])) \
                              * (1 + (eurostat_products['AP_EI'] - (eurostat_products['AP_DHO']/0.9)) / (eurostat_products['MAP_EI'] - (eurostat_products['MAP_DHO']/0.9)))  # The final units are Tg CO2.
elec_emission_incl_autoprod[elec_emission_incl_autoprod<0] = 0
elec_emission_incl_autoprod.name = 'electricity-only emissions incl autoprod'

In [12]:
elec_emission_incl_autoprod.head()

country  year
AT       1990    10.4819
         1991    11.0529
         1992    8.08414
         1993    7.92401
         1994      8.199
Name: electricity-only emissions incl autoprod, dtype: object

In [13]:
## Save results to files
elec_emission.to_csv(path_output_elec_emis)
elec_emission_incl_autoprod.to_csv(path_output_elec_emis_autoprod)